In [26]:
#%pip install keystoneauth1 --user
%pip install swiftclient --user

#%pip install pandas --user
import os
import pandas as pd
import pickle
import random
import keystoneauth1
#import swiftclient
import warnings
warnings.filterwarnings('ignore')
from pymongo import MongoClient
from keystoneauth1 import session
from keystoneauth1.identity import v3
#from swiftclient.client import Connection, ClientException

ERROR: Could not find a version that satisfies the requirement swiftclient (from versions: none)
ERROR: No matching distribution found for swiftclient
Note: you may need to restart the kernel to use updated packages.


In [27]:
db_config= {
"MONGODB_ADDRESS"    : os.environ.get('KEY_MONGO_ADDRESS','mongodb-service.stag-data.svc.cluster.local'),
"MONGODB_PORT"       : os.environ.get('KEY_MONGO_PORT', '27017'),
"MONGODB_USERNAME"   : os.environ.get('KEY_MONGO_USER', 'IBSO-read-write'),
"MONGODB_PASSWORD"   : os.environ.get('KEY_MONGO_PASSWORD', 'NHWD5bw0xcMgy1Y'),
"MONGODB_DATABASE"   : os.environ.get('KEY_MONGO_DATABASE', 'admin')
}
mongo_client = MongoClient(
                ('mongodb://%s:%s@' + db_config["MONGODB_ADDRESS"] + ':' + db_config["MONGODB_PORT"] + '/' + db_config["MONGODB_DATABASE"]) % (
                    db_config["MONGODB_USERNAME"], db_config["MONGODB_PASSWORD"]), replicaset='rs0')
infodocs_metadata = mongo_client["IBSO"]["infodocs_metadata"]

In [4]:
%run InfodocsLoader.py
%run MongoClass.py
#%run helpers.py

In [13]:
dataAccessObj = MongoDataAccess(db_config,'IBSO')
metadata_collection = dataAccessObj.connect_mongo()["IBSO"]["infodocs_metadata"]

In [31]:
resultsDf = pd.read_csv('Validation_Customer_Solution_Matching_v2.csv',dtype = {'filtered_solns':str})

In [32]:
resultsDf

,customer_no,installation_no,System No,filtered_solns,Solution Title,Solution App Component,Is the Solution Relevant?,Top-k Relevant Solutions (proposed & missed) sorted by relevancy,Comment
0,490533,20848565,NaN,455942,NaN,NaN,NaN,NaN,NaN
1,490533,20848565,NaN,2585851,NaN,NaN,NaN,NaN,NaN
2,490533,20848565,NaN,3154651,NaN,NaN,NaN,NaN,NaN
3,530444,20848565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,530444,20848565,NaN,3148891,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
128,1610858,90685866,NaN,2814277,NaN,NaN,NaN,NaN,NaN
129,1610858,90685866,NaN,2748224,NaN,NaN,NaN,NaN,NaN
130,1610858,90685866,NaN,3147821,NaN,NaN,NaN,NaN,NaN
131,1610858,90685866,NaN,2978817,NaN,NaN,NaN,NaN,NaN


In [33]:
resultsDf['customer_no_2'] = resultsDf['customer_no'].astype(str).apply(lambda x: '0' * (10-len(x)) + x )
resultsDf['installation_no_2'] = resultsDf['installation_no'].astype(str).apply(lambda x: '0' * (10-len(x)) + x )

In [34]:
infDf = pd.DataFrame(metadata_collection.find({'incident_created_at_dt': {
    '$gt': datetime(2020,1,1),
    '$lt': datetime(2022,4,26)
},'customer_no': {'$in':list(resultsDf['customer_no_2'].astype(str).unique())}}))

NameError: name 'metadata_collection' is not defined

In [19]:
infDf = infDf.explode('solns')

In [20]:
infDf.reset_index(inplace = True)

In [21]:
def findSolution(row):
    cno = row['customer_no_2']
    inst = row['installation_no_2']
    soln = row['filtered_solns']
    subsetDf = infDf.loc[(infDf['customer_no'] == cno) & (infDf['installation_no'] == inst)]
    return soln in subsetDf['solns'].values

In [22]:
resultsDf['Solution Presented to Customer Before'] = resultsDf.apply(findSolution,axis = 1)

In [23]:
resultsDf.loc[resultsDf['Solution Presented to Customer Before']]

,customer_no,installation_no,System No,filtered_solns,Solution Title,Solution App Component,Is the Solution Relevant?,Top-k Relevant Solutions (proposed & missed) sorted by relevancy,Comment,customer_no_2,installation_no_2,Solution Presented to Customer Before
0,490533,20848565,NaN,455942,NaN,NaN,NaN,NaN,NaN,0000490533,0020848565,True
14,1741934,90439993,NaN,2963059,NaN,NaN,NaN,NaN,NaN,0001741934,0090439993,True
23,2348832,90463694,NaN,2963059,NaN,NaN,NaN,NaN,NaN,0002348832,0090463694,True
33,707397,90451413,NaN,2963059,NaN,NaN,NaN,NaN,NaN,0000707397,0090451413,True
42,707397,90451413,NaN,2954782,NaN,NaN,NaN,NaN,NaN,0000707397,0090451413,True
43,1721995,90449777,NaN,2963059,NaN,NaN,NaN,NaN,NaN,0001721995,0090449777,True
48,1721995,90449777,NaN,2812948,NaN,NaN,NaN,NaN,NaN,0001721995,0090449777,True
50,1721995,90449777,NaN,2434549,NaN,NaN,NaN,NaN,NaN,0001721995,0090449777,True
53,12999,90320393,NaN,2963059,NaN,NaN,NaN,NaN,NaN,0000012999,0090320393,True
73,2054244,90445910,NaN,2963059,NaN,NaN,NaN,NaN,NaN,0002054244,0090445910,True


In [24]:
solutions_collection = mongo_client["main"]["solutions_parsed_version"]

In [25]:
solutionDf = pd.DataFrame(solutions_collection.find({'Number':{'$in' :list(resultsDf['filtered_solns'].astype(str).unique())}}))

In [26]:
solutionDf.columns

Index(['_id', 'AbapCode', 'Attributes', 'Attributes_uh', 'Category',
       'ChangedOn', 'ChangedOn_dt', 'CorrectionInstructionsObjectList',
       'CreatedOn', 'CreatedOn_dt', 'CurrentStatus', 'ExpirationDate',
       'HasPreconditions', 'InternalMemo', 'Languages', 'Number',
       'OtherSAPComponents', 'Patches', 'PrimarySAPComponent', 'Priority',
       'Product', 'RatingInfo', 'RefNumber', 'ReferenceTo', 'ReferencedBy',
       'ReleasedOn', 'ReleasedOn_dt', 'SoftwareComponents', 'SupportPackages',
       'Tags', 'TargetStatus', 'TextSections', 'Title', 'TranslationFlag',
       'Type', 'Version', 'Visibility', 'extracted_at', 'extracted_at_dt',
       'script', 'sha_hashed', 'Symptom', 'Solution', 'Reason&Prerequisites',
       'KeyTerms', 'ReproducingTheIssue', 'Cause', 'AbapCode_parsed',
       'NormalizedTitle', 'NormalizedSymptom', 'NormalizedSolution',
       'NormalizedReason&Prerequisites', 'NormalizedKeyTerms',
       'NormalizedReproducingTheIssue', 'NormalizedCause', 'No

In [27]:
solutionDf = solutionDf.sort_values('Version').drop_duplicates('Number',keep = 'last')

In [3]:
%pip install nltk --user
%pip install bs4 --user

from bs4 import BeautifulSoup


import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from helpers import NgramPreprocessor
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 128 kB 11.4 MB/s            
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=2245 sha256=e5f69bed9364c8745e6fa61da41d7bc814c3eeec76dd67cb87bb5eb0f3ba674a
  Stored in directory: /home/jovyan/.cache/pip/wheels/19/f5/6d/a97dd4f22376d4472d5f4c76c7646876052ff3166b3cf71050
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [5]:
ng = NgramPreprocessor()

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/jovyan/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
def update_nested_ctr(ctr_dict, key1, key2): 
    if key1 in ctr_dict:
        ctr_dict[key1][key2] = ctr_dict[key1].get(key2, 0) + 1
    else:
        ctr_dict[key1] = {key2: 1}

In [10]:
class phrase_detector:
    def __init__(self, delimiter='#'):
        """
        sentences_gen: iterator of list of words in a sentence
        min_count: phrases under this amount of occurrences will not be kept in the vocab
        threshold: phrases that appeared this amount of times will become phrase no matter what
        """
        self.all_phrases = set()
        self.delimiter = delimiter
        self.keeper = dict()
        self.reverse_keeper = dict()
        self.phrases = dict()
        self.next_words = dict()
        self.fiori_names = set()
        self.formatter = NgramPreprocessor()
        fiori_app_name = pd.read_csv("fiori_dump.csv",usecols=['AppName'])
        self.fiori_app_name = fiori_app_name['AppName'].tolist()
        self.add_fiori_phrases()
        self.fiori_names = dict()
        self.transform_fiori_names()

    
    def add_fiori_phrases(self):
        self.add_phrases(list_phrases=self.fiori_app_name,formatter = self.formatter)
        print("fiori application name imported")

    def transform_fiori_names(self):
        for text in self.fiori_app_name:
            cleaned_text = self.formatter.tokenize(text, corpus_only=False)
            transform_fiori = self.detect_phrases(cleaned_text)
            transform_fiori.sort(key = lambda x:len(x),reverse = True)
            if transform_fiori:
                self.fiori_names[transform_fiori[0]] = text
        print("fiori application name transformed")
            
    def tokenize(self,formatter,text):
        texts_clean = formatter.tokenize(text, corpus_only=False) # list
        return texts_clean

    def cut_phrases(self,phrases):
        phrases_length = len(phrases)
        if phrases_length>4:
            phrases = phrases[:4]
        return phrases

    def import_custom_phases(self,phases):
        word = phases[0]
        word_after = phases[1]
        self.next_words[word] = self.next_words.get(word, set()) | {word_after}
        
    def add_phrases(self,list_phrases,formatter):
        for phrases in list_phrases:
            cut_phrases = self.cut_phrases(self.tokenize(self.formatter,phrases))
            length_phrases = len(cut_phrases)
            if length_phrases <= 1:
                pass
            elif length_phrases == 2:
                self.import_custom_phases(cut_phrases)
            elif length_phrases == 3:
                self.import_custom_phases(cut_phrases[0:2])
                self.import_custom_phases([''.join(cut_phrases[0:2]),''.join(cut_phrases[2])])
            elif length_phrases == 4:
                self.import_custom_phases(cut_phrases[0:2])
                self.import_custom_phases([''.join(cut_phrases[0:2]),''.join(cut_phrases[2])])
                self.import_custom_phases([''.join(cut_phrases[0:3]),''.join(cut_phrases[3])])
    
    def detect_phrases(self, sentence:list) -> list:
        transformed = []
        idx = 0
        while idx < len(sentence):
            word = sentence[idx]
            try:
                word_after = sentence[idx+1]
            except IndexError:
                # transformed.append(word) # reaches the last element
                break
            if word in self.next_words and word_after in self.next_words[word]:
                bi_phrase = word + word_after
                try:
                    two_word_after = sentence[idx+2]
                except IndexError:
                    transformed.append(bi_phrase)
                    break
                if bi_phrase in self.next_words and two_word_after in self.next_words[bi_phrase]:
                    tri_phrase = bi_phrase + two_word_after
                    try:
                        three_word_after = sentence[idx+3]
                    except IndexError:
                        transformed.append(tri_phrase)
                        break
                    if tri_phrase in self.next_words and three_word_after in self.next_words[tri_phrase]:
                        four_phrase = tri_phrase+three_word_after
                        transformed.append(four_phrase)
                    else:
                        transformed.append(tri_phrase)
                else:
                    transformed.append(bi_phrase)
                idx += 1 # make sure the next word could also form a phrase with next next word
            else:
                # transformed.append(word)
                idx += 1
                continue
#         print("trigram transformed",transformed)
        return transformed
    
    def detect_fiori_name(self, text):
        fiori = set()
        cleaned_text = self.formatter.tokenize(text, corpus_only=False)
        phrases = self.detect_phrases(cleaned_text)
        if phrases:
            for i in phrases:
                if i in self.fiori_names:
                    fiori.add(self.fiori_names[i])
        return fiori


In [15]:
phd = phrase_detector()

LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - '/home/jovyan/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [18]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...


True

In [19]:
phd = phrase_detector()

/home/jovyan/.local/lib/python3.6/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


fiori application name imported
fiori application name transformed


In [20]:
phd.next_words

{'interface': {'monitor'},
 'interfacemonitor': {'application'},
 'interfacemonitorapplication': {'interface'},
 'configure': {'allocation',
  'alternative',
  'bop',
  'consumption',
  'custom',
  'data',
  'master',
  'order',
  'personalize',
  'rule',
  'substitution',
  'supply',
  'value'},
 'configurevalue': {'map'},
 'message': {'dashboard', 'monitor'},
 'messagedashboard': {'application'},
 'messagedashboardapplication': {'interface'},
 'connector': {'monitor'},
 'display': {'account',
  'accrual',
  'acquisition',
  'activity',
  'actual',
  'adjustment',
  'allocation',
  'amortisation',
  'archive',
  'authorization',
  'available',
  'batch',
  'batchspecific',
  'bill',
  'billable',
  'bin',
  'block',
  'bom',
  'bp',
  'business',
  'cancel',
  'capacity',
  'cash',
  'change',
  'characteristic',
  'class',
  'classify',
  'code',
  'collection',
  'communication',
  'compliance',
  'condition',
  'configuration',
  'connectivity',
  'consumption',
  'contract',
  'co

In [21]:
phd.fiori_app_name

['Interface Monitor - SAP Application Interface Framework',
 'Configure Value Mapping',
 'Message Dashboard - SAP Application Interface Framework',
 'Connector Monitor',
 'Display Account Card - Romania',
 'Display GL w. Corresp. Acct. - Romania',
 'Analyze Year End Postings - Romania',
 'Cash Journal - Romania',
 'Run Month/Year End Postings - Romania',
 'Display Inventory Register - Romania',
 'Analyze Document Journal - Romania',
 'Merge VIES 394 Files - Romania',
 'ODN Generator - Romania',
 'Romanian Storage Sheet - Romania',
 'Analytical Material Balance - Romania',
 'Core configurator - create IMG',
 'Core configurator - maintain IMG',
 'Manage Providers of Information',
 'Manage Intrastat Declarations',
 'Display Migrated Trans. for Deletion - Japan',
 'Delete Migrated Asset Transaction - Japan',
 'Add Asset Trans. as Migration - Japan',
 'Delete LYs Migrated Asset - Japan',
 'Migrate Last Years Asset Data - Japan',
 'Migrate Property Tax Data - XML - Japan',
 'Create Change Re

In [28]:
solutions_collection = mongo_client["main"]["solutions_parsed_version"]

In [35]:
solutionDf = pd.DataFrame(solutions_collection.find({'Number':{'$in' :list(resultsDf['filtered_solns'].astype(str).unique())}}))

In [36]:
solutionDf.columns

Index(['_id', 'AbapCode', 'Attributes', 'Attributes_uh', 'Category',
       'ChangedOn', 'ChangedOn_dt', 'CorrectionInstructionsObjectList',
       'CreatedOn', 'CreatedOn_dt', 'CurrentStatus', 'ExpirationDate',
       'HasPreconditions', 'InternalMemo', 'Languages', 'Number',
       'OtherSAPComponents', 'Patches', 'PrimarySAPComponent', 'Priority',
       'Product', 'RatingInfo', 'RefNumber', 'ReferenceTo', 'ReferencedBy',
       'ReleasedOn', 'ReleasedOn_dt', 'SoftwareComponents', 'SupportPackages',
       'Tags', 'TargetStatus', 'TextSections', 'Title', 'TranslationFlag',
       'Type', 'Version', 'Visibility', 'extracted_at', 'extracted_at_dt',
       'script', 'sha_hashed', 'Symptom', 'Solution', 'Reason&Prerequisites',
       'KeyTerms', 'ReproducingTheIssue', 'Cause', 'AbapCode_parsed',
       'NormalizedTitle', 'NormalizedSymptom', 'NormalizedSolution',
       'NormalizedReason&Prerequisites', 'NormalizedKeyTerms',
       'NormalizedReproducingTheIssue', 'NormalizedCause', 'No